In [1]:
import os
import requests
import json
import sys
import time
import pathlib  
from requests.auth import HTTPBasicAuth

In [2]:
def read_geom(file_path):
    with open(file_path) as f:
        data = json.load(f)
        if 'features' in data and data['features']:
            bbox = data['features'][0]['geometry']
            return bbox
        else:
            print(f"No features found in {file_path}. Skipping this file.")
            return None
    
def place_order(request, auth):
    response = requests.post('https://api.planet.com/compute/ops/orders/v2', data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    try:
        order_id = response.json()['id']
        print(order_id)
        order_url = 'https://api.planet.com/compute/ops/orders/v2' + '/' + order_id
        return order_url
    except KeyError:
        print(f"Failed to place order for {request['name']}. ID not found.")
        return None

def poll_for_success(order_url, auth):
    while True:
        r = requests.get(order_url, auth=(api_key, ""))
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)

def download_results(results, geojson_file, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
  
    parking_lot_number = os.path.splitext(geojson_file)[0]
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Tree_Nation_Polygons/tree_nation_small_size_polygons/small_sizepolys', parking_lot_number, name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            path.parent.mkdir(parents=True, exist_ok=True) 
            r = requests.get(url, allow_redirects=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

geojson_folder = '/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/FirstTreePlanted_new'
geojson_files = [f for f in os.listdir(geojson_folder) if f.endswith('.geojson')]

In [3]:
API_keys = []

In [9]:
import os
import sys
import requests
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

try:
    os.environ['PL_API_KEY'] = 'PLAK731e474fdd584ac2a0ecb22def40e20d'
    api_key = os.getenv('PL_API_KEY')
except Exception as e:
    print("Failed")
    sys.exit()

headers = {'Content-Type':'application/json'}
geojson_folder = '/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Tree_Nation_Polygons/tree_nation_small_size_polygons/small_sizepolys'


geojson_files = [f for f in os.listdir(geojson_folder) if f.endswith('.geojson')]


for geojson_file in geojson_files:
    print(f"Processing {geojson_file}...")
    
    geojson_path = os.path.join(geojson_folder, geojson_file)
    
    clip_aoi = read_geom(geojson_path)
    if clip_aoi is None:
        continue

 

  
    clip = {
        "clip": {
            "aoi": clip_aoi
        }
    }


    geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": clip_aoi
    }

    # Read the GeoJSON file
    with open(geojson_path, 'r') as f:
        data = json.load(f)

  
    if 'features' in data and data['features']:
     
            
            if 'properties' in feature and feature['properties'].get('created_at'):
                created_at = parse(feature['properties']['created_at'])
                gte_date = created_at + relativedelta(years=5)
                lte_date = gte_date + relativedelta(months=1)
                
                date_range_filter = {
                    "type": "DateRangeFilter",
                    "field_name": "acquired",
                    "config": {
                        "gte": gte_date.isoformat(),
                        "lte": lte_date.isoformat()
                        }
                }
            else:
                print(f"'created_at' not found or is None in feature. Skipping this feature.")
                continue

           
            cloud_cover_filter = {
            "type": "RangeFilter",
            "field_name": "cloud_cover",
            "config": {
                "lt": 0.05
            }
            }

           
            combined_filter = {
            "type": "AndFilter",
            "config": [geometry_filter, date_range_filter, cloud_cover_filter]
            }

            item_type = "PSScene"
            # API request object
            search_request = {
            "item_types": [item_type], 
            "filter": combined_filter
            }

            search_result = requests.post(
                'https://api.planet.com/data/v1/quick-search',
                auth=HTTPBasicAuth(api_key, ''),
                json=search_request)
            
          
            if search_result.status_code != 200:
                print(f"Request failed with status code {search_result.status_code}. Skipping this feature.")
                print(search_result.content)
                continue

            geojson = search_result.json()

            if 'features' in geojson and geojson['features']:
                
                image_ids = [feature['id'] for feature in geojson['features']]
            else:
                print(f"No images found that match the filters. Skipping this feature.")
                continue

            
            request = {
                "name": os.path.splitext(geojson_file)[0], 
                "products": [
                    {
                        "item_ids": image_ids,
                        "item_type": "PSScene",
                        "product_bundle": "visual",
                    }
                ],
                "tools": [clip]
            }
            print(request)

            order_url = place_order(request, auth=(api_key, ""))
            if order_url is not None:
                print(order_url)
                poll_for_success(order_url, api_key)
                r = requests.get(order_url, auth=(api_key, ""))
                response = r.json()
                results = response['_links']['results']
                print(results)                                                    

                download_results(results, geojson_file)

                print(f"Processing of {geojson_file} complete.")
            else:
                print(f"Skipping download for {geojson_file} due to failed order placement.")

        
    else:
        print(f"No features found in {geojson_file}. Skipping this file.")

print("Processing of all GeoJSON files complete.")

Processing 393.geojson...
No images found that match the filters. Skipping this feature.
Processing 841.geojson...
No images found that match the filters. Skipping this feature.
Processing 60.geojson...
No images found that match the filters. Skipping this feature.
Processing 20.geojson...
{'name': '20', 'products': [{'item_ids': ['20210521_110022_1105', '20210521_110020_1105', '20210520_103921_1010', '20210519_103821_101b', '20210518_102014_35_2259', '20210518_102012_06_2259', '20210518_103718_1012', '20210518_103717_1012', '20210516_101302_48_2428', '20210516_103608_103c', '20210514_103727_1039', '20210514_103728_1039', '20210514_110222_82_227b', '20210508_102320_67_2259', '20210508_104027_1008', '20210508_104026_1008', '20210507_101049_66_2420', '20210507_112308_27_105d', '20210507_112306_77_105d', '20210506_101011_26_2453', '20210505_110416_58_2274', '20210505_110207_50_225b', '20210505_101421_82_2463', '20210505_101419_48_2463', '20210504_110322_04_2426', '20210504_110319_74_2426'